In [ ]:
import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
import matplotlib.animation as animation
from matplotlib.colors import LightSource
from matplotlib import cm
from matplotlib.gridspec import GridSpec

# Import colormaps and limits
import sys
sys.path.append('./')
from colormaps import *

In [52]:
data_dir = '../../Zenodo/model_output'

la_M_IM = xr.open_dataset(f'{data_dir}/LA_M.nc')
la_H_IM = xr.open_dataset(f'{data_dir}/LA_H.nc')
la_M_QU = xr.open_dataset(f'{data_dir}/QU_M.nc')

la_M_laddie = xr.open_dataset(f'{data_dir}/LA_M_laddie_fields.nc')
la_H_laddie = xr.open_dataset(f'{data_dir}/LA_H_laddie_fields.nc')

spinup = xr.open_dataset(f'{data_dir}/spinup.nc')  

In [ ]:
# Plot LADDIE MODERATE-MELT (LA_M) fields

# Select time slices and x extent
times = [0,200,1000]
xslicei = [10000,-100000,-250000]
printxy = []

# Create plot and axes for subplots
fig = plt.figure(figsize=(10,16), dpi=200)
gs = GridSpec(5, 4, height_ratios=[240000/240000, 350000/240000, 500000/240000, .1, .2],hspace=0.1, wspace=0.2)

# t = 0
ax1 = fig.add_subplot(gs[0])                            
ax2 = fig.add_subplot(gs[1], sharex=ax1, sharey=ax1)
ax3 = fig.add_subplot(gs[2], sharex=ax1, sharey=ax1)
ax4 = fig.add_subplot(gs[3], sharex=ax1, sharey=ax1)
                         
# t = 10
ax5 = fig.add_subplot(gs[4])                            
ax6 = fig.add_subplot(gs[5], sharex=ax1, sharey=ax5)
ax7 = fig.add_subplot(gs[6], sharex=ax1, sharey=ax5)
ax8 = fig.add_subplot(gs[7], sharex=ax1, sharey=ax5)

# t = 200
ax9 = fig.add_subplot(gs[8])                            
ax10 = fig.add_subplot(gs[9], sharex=ax1, sharey=ax9)
ax11 = fig.add_subplot(gs[10], sharex=ax1, sharey=ax9)
ax12 = fig.add_subplot(gs[11], sharex=ax1, sharey=ax9)

# cax for melt (m), velocity (v), and draft (d), and ice velocities (i)
caxm = fig.add_subplot(gs[17])  
caxv = fig.add_subplot(gs[18])  
caxd = fig.add_subplot(gs[16])  
caxi = fig.add_subplot(gs[19])  

# Turn ticklabels off for specific axes
for axes in [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10, ax11,ax12]:
    plt.setp(axes.get_xticklabels(), visible=False)
for axes in [ax2,ax3,ax4, ax6, ax7, ax8, ax10, ax11,ax12]:
    plt.setp(axes.get_yticklabels(), visible=False)

# List of axes for topview plots
axs = [[ax1, ax2, ax3, ax4], [ax5, ax6, ax7, ax8], [ax9, ax10, ax11, ax12]]

# Loop over time slices
for i, t in enumerate(times):

    # Select time slice and x slice for laddie file (ds) and IMAUICE file (dsIM)
    ds          = la_M_laddie.sel(time=t, x = slice(xslicei[i],250000))
    dsIM        = la_M_IM.sel(time=t, x = slice(xslicei[i],250000))

    # Transpose data
    newx        = ds.y[::-1]/1000
    newy        = ds.x/1000
    newUt       = -ds.Vt.T
    newVt       = ds.Ut.T
    
    newdraft    = dsIM.Hs.T-dsIM.Hi_eff_cf.T
    newmelt     = dsIM.BMB.T*-1
    newmask     = dsIM.mask.T
    newmaskgl   = xr.where(dsIM['mask']==7, 1,0).T

    if t==0: # u_surf and v_surf are not written to t = 0, so take next time step
        dsIM0 = spinup.sel(time=50000, x = slice(xslicei[i],250000)) #la_M_IM.sel(time=t+.125, x = slice(xslicei[i],250000))
        newicevelu = -dsIM0['v_surf'].T
        newicevelv = dsIM0['u_surf'].T
    else:
        newicevelu = -dsIM['v_surf'].T
        newicevelv = dsIM['u_surf'].T

    # Compute deep melt average [m/yr] and integrated melt flux [Gt/yr]
    deepmelt    = np.nanmean((xr.where(np.logical_and(dsIM.mask==4, dsIM.Hs - dsIM.Hi < -300), 1, np.nan) * - dsIM.BMB).T)
    intmelt     = np.nansum(newmelt * 2000 * 2000 * 918 * 1e-12) # melt in m ice per year * area per gridcell * rhoice * 1e-12 [conversion kg to Gt]

    ### COLUMN 1: DRAFT
    # Plot draft
    imdraft = axs[i][0].pcolormesh( newx, newy, newdraft, cmap=cmap_draft, norm =norm_draft)

    # Add contours
    # CD = axs[i][0].contour(newx, newy, newdraft,  [-300, -275, -250, -225, -200, -175, -150, -125, -100], cmap='inferno_r',zorder=1)
    # h, l = CD.legend_elements()

    axs[i][0].contour(newx, newy, -newdraft,  [100, 125, 150, 175, 200, 225, 250, 300], colors='black', linewidths=3, zorder=1)
    CD = axs[i][0].contour(newx, newy, -newdraft,  [100, 125, 150, 175, 200, 225, 250, 300], cmap='cmo.deep',  zorder=1)
    h, l = CD.legend_elements()

    # Set labels, and add year
    print((250000-xslicei[i])/2000 + xslicei[i]/1000)
    axs[i][0].set_ylabel('[km]', fontsize=11)
    axs[i][0].text(-90, (250000-xslicei[i])/2000 + xslicei[i]/1000, f'Model year {t}', fontsize=13,rotation=90,horizontalalignment='center',verticalalignment='center')
    axs[i][0].set_yticks([-250,-200,-150, -100,-50, 0, 50, 100, 150, 200, 250])
    
    ### COLUMN 2: MELT
    # Plot melt
    immelt = axs[i][1].pcolormesh( newx, newy, newmelt, cmap=cmap_BMB, norm = norm_BMB)

    # Print deepmelt and intmelt
    axs[i][1].text(38, xslicei[i]/1000+25, f'{deepmelt:.2f} m/yr', fontsize=11, horizontalalignment='right')
    axs[i][1].text(38, xslicei[i]/1000+5, f' {intmelt:.2f} Gt/yr', fontsize=11, horizontalalignment='right')

    print(f'max melt = {np.nanmax(xr.where(newdraft<-300, newmelt, np.nan).values)}')
    
    # Plot grounded and ocean for both melt and draft
    for j in range(2):
        axs[i][j].pcolormesh(newx, newy,  xr.where(newmask==1, 1, np.nan),      cmap=cmap_ocean,    norm=norm_ocean)
        axs[i][j].pcolormesh(newx, newy,    xr.where(np.logical_or(newmask==3,newmask==7), 1, np.nan),   cmap=cmap_ground,   norm=norm_ground)        

    ### COLUMN 3: PLUME VELOCITY
    # Plot plume velocity
    imvel = axs[i][2].pcolormesh( newx, newy, np.sqrt(newUt**2+newVt**2), cmap=cmap_u_lad, norm = norm_u_lad)
    # Mask ocean and grounded
    axs[i][2].pcolormesh(newx, newy,  xr.where(newmask==1, 1, np.nan),      cmap=cmap_ocean,    norm=norm_ocean)    
    axs[i][2].pcolormesh(newx, newy,    xr.where(np.logical_or(newmask==3,newmask==7), 1, np.nan),   cmap=cmap_ground,   norm=norm_ground)        
    
    # Show velocity vectors
    stepx = 5   # x spacing vectors
    stepy = 10  # y spacing vectors 
    axs[i][2].quiver(newx[3::stepx], newy[::stepy], newUt[::stepy,3::stepx], newVt[::stepy, 3::stepx], scale=0.5, width=1/75, headlength=5, headaxislength=4, headwidth=3, facecolor='k')


    ### COLUMN 4: ICE VELOCITY
    imvelice = axs[i][3].pcolormesh( newx, newy, np.sqrt(newicevelu**2+newicevelv**2), cmap=cmap_u_surf, norm = norm_u_surf)
    
    # Mask ocean and GL
    axs[i][3].pcolormesh(newx, newy,  xr.where(newmask==1, 1, np.nan),      cmap=cmap_ocean,    norm=norm_ocean)        
    axs[i][3].pcolormesh(newx, newy,  xr.where(newmaskgl==1, 1, np.nan),      cmap=cmap_GL,    norm=norm_GL)

    # Show velocity vectors
    stepx = 5   # x spacing vectors
    stepy = 40  # y spacing vectors 
    axs[i][3].quiver(newx[::stepx], newy[::stepy], newicevelu[::stepy,::stepx], newicevelv[::stepy, ::stepx], scale=4000, width=1/100, facecolor='white')


# Set ticks and labels
for j in range(4):
    axs[2][j].set_xticks([-40,0,40])
    axs[1][j].set_xticks([-40,0,40])
    axs[0][j].set_xticks([-40,0,40])
    axs[2][j].tick_params(axis='both', labelsize=12)
    axs[1][j].tick_params(axis='both', labelsize=12)
    axs[0][j].tick_params(axis='both', labelsize=12)
    axs[2][j].xaxis.set_tick_params(labelbottom=True,labelcolor="k",)
    axs[2][j].set_xlabel('[km]', fontsize=11)

    # Set aspect
    axs[2][j].set_aspect(0.5)
    axs[1][j].set_aspect(0.5)
    axs[0][j].set_aspect(0.5)

# Take colors at regular intervals spanning the colormap.
colors = cmap_ground(np.linspace(0, 1, 17))
cplot=colors[0]

# Pannel labels
lab0 = ['(a)', '(b)', '(c)', '(d)']
lab200 = ['(e)', '(f)', '(g)', '(h)']
lab1000 = ['(i)', '(j)', '(k)', '(l)']
for j in range(3):
    axs[0][j].text(-37,20,lab0[j],fontsize=12,backgroundcolor=cplot,weight='bold')
    axs[1][j].text(-37,-90,lab200[j],fontsize=12,backgroundcolor=cplot,weight='bold')
    axs[2][j].text(-37,-240,lab1000[j],fontsize=12,backgroundcolor=cplot,weight='bold')

axs[0][3].text(-37,20,lab0[3],fontsize=12,c='white', weight='bold')
axs[1][3].text(-37,-90,lab200[3],fontsize=12,c='white', weight='bold')
axs[2][3].text(-37,-240,lab1000[3],fontsize=12,c='white', weight='bold')


# Set colorbars
cbar = plt.colorbar(immelt, ax = caxm, ticks=[-10,-1, 0, 1, 10, 100], label='sub-shelf melt [m/yr]', extend='both', orientation='horizontal', pad=.8, fraction=1)
cbar.ax.set_xticklabels([-10,-1, 0, 1, 10, 100], fontsize=11);
cbar.set_label('Sub-shelf melt \n[m/yr]', fontsize=11)

cbar = plt.colorbar(imvel,     ax = caxv, orientation='horizontal', label='layer velocity [m/s]',extend='max',  pad=.1, fraction=1,location='bottom')
cbar.ax.tick_params(labelsize=11)
cbar.set_label('Layer velocity \n[m/s]', fontsize=11)

cbar = plt.colorbar(imdraft,   ax = caxd, orientation='horizontal', label='draft [m]', extend='min', pad=.1, fraction=1)
cbar.ax.tick_params(labelsize=11)
cbar.set_label('Ice draft \n[m]', fontsize=11)

cbar = plt.colorbar(imvelice,   ax = caxi, orientation='horizontal', label='ice surface velocity [m/yr]', extend='max', pad=.1, fraction=1)
cbar.ax.tick_params(labelsize=11)
cbar.set_label('Ice surface velocity \n[m/yr]', fontsize=11)


caxm.set_axis_off()
caxv.set_axis_off()
caxd.set_axis_off()
caxi.set_axis_off()

for ax in axs[2]:
    ax.set_aspect(0.5)

# Set draft contour legend
axs[0][3].legend(h,['-100 m', '-125 m', '-150 m', '-175 m', '-200 m', '-225 m', '-250 m', '-300 m'], fontsize=11, bbox_to_anchor=(1.1, 1),loc=2)

axs[0][0].text(-34  , 90,'GL'   , size=12 ,weight='bold',ha='center', va='center')
axs[0][0].text(-29  , 80,'w'    , size=9 ,weight='bold',ha='center', va='center')
axs[0][0].text( 33  , 90,'GL'   , size=12 ,weight='bold',ha='center', va='center')
axs[0][0].text( 38  , 80,'e'    , size=9 ,weight='bold',ha='center', va='center')



fig.align_ylabels() 

fig.savefig('fig04.pdf',dpi=300,bbox_inches='tight')

In [ ]:
# Plot LADDIE HIGH-MELT (LA_H) fields

# Select time slices and x extent
times = [0,200,1000]
xslicei = [10000,-100000,-250000]
printxy = []

# Create plot and axes for subplots
fig = plt.figure(figsize=(10,16), dpi=200)
gs = GridSpec(5, 4, height_ratios=[240000/240000, 350000/240000, 500000/240000, .1, .2],hspace=0.1, wspace=0.2)

# t = 0
ax1 = fig.add_subplot(gs[0])                            
ax2 = fig.add_subplot(gs[1], sharex=ax1, sharey=ax1)
ax3 = fig.add_subplot(gs[2], sharex=ax1, sharey=ax1)
ax4 = fig.add_subplot(gs[3], sharex=ax1, sharey=ax1)
                         
# t = 10
ax5 = fig.add_subplot(gs[4])                            
ax6 = fig.add_subplot(gs[5], sharex=ax1, sharey=ax5)
ax7 = fig.add_subplot(gs[6], sharex=ax1, sharey=ax5)
ax8 = fig.add_subplot(gs[7], sharex=ax1, sharey=ax5)

# t = 200
ax9 = fig.add_subplot(gs[8])                            
ax10 = fig.add_subplot(gs[9], sharex=ax1, sharey=ax9)
ax11 = fig.add_subplot(gs[10], sharex=ax1, sharey=ax9)
ax12 = fig.add_subplot(gs[11], sharex=ax1, sharey=ax9)

# cax for melt (m), velocity (v), and draft (d), and ice velocities (i)
caxm = fig.add_subplot(gs[17])  
caxv = fig.add_subplot(gs[18])  
caxd = fig.add_subplot(gs[16])  
caxi = fig.add_subplot(gs[19])  

# Turn ticklabels off for specific axes
for axes in [ax1,ax2,ax3,ax4,ax5,ax6,ax7,ax8,ax9,ax10, ax11,ax12]:
    plt.setp(axes.get_xticklabels(), visible=False)
for axes in [ax2,ax3,ax4, ax6, ax7, ax8, ax10, ax11,ax12]:
    plt.setp(axes.get_yticklabels(), visible=False)

# List of axes for topview plots
axs = [[ax1, ax2, ax3, ax4], [ax5, ax6, ax7, ax8], [ax9, ax10, ax11, ax12]]

# Loop over time slices
for i, t in enumerate(times):

    # Select time slice and x slice for laddie file (ds) and IMAUICE file (dsIM)
    ds          = la_H_laddie.sel(time=t, x = slice(xslicei[i],250000))
    dsIM        = la_H_IM.sel(time=t, x = slice(xslicei[i],250000))

    # Transpose data
    newx        = ds.y[::-1]/1000
    newy        = ds.x/1000
    newUt       = -ds.Vt.T
    newVt       = ds.Ut.T
    
    newdraft    = dsIM.Hs.T-dsIM.Hi_eff_cf.T
    newmelt     = dsIM.BMB.T*-1
    newmask     = dsIM.mask.T
    newmaskgl   = xr.where(dsIM['mask']==7, 1,0).T

    if t==0: # u_surf and v_surf are not written to t = 0, so take next time step
        dsIM0 = spinup.sel(time=50000, x = slice(xslicei[i],250000)) #la_M_IM.sel(time=t+.125, x = slice(xslicei[i],250000))
        newicevelu = -dsIM0['v_surf'].T
        newicevelv = dsIM0['u_surf'].T
    else:
        newicevelu = -dsIM['v_surf'].T
        newicevelv = dsIM['u_surf'].T

    # Compute deep melt average [m/yr] and integrated melt flux [Gt/yr]
    deepmelt    = np.nanmean((xr.where(np.logical_and(dsIM.mask==4, dsIM.Hs - dsIM.Hi < -300), 1, np.nan) * - dsIM.BMB).T)
    intmelt     = np.nansum(newmelt * 2000 * 2000 * 918 * 1e-12) # melt in m ice per year * area per gridcell * rhoice * 1e-12 [conversion kg to Gt]

    ### COLUMN 1: DRAFT
    # Plot draft
    imdraft = axs[i][0].pcolormesh( newx, newy, newdraft, cmap=cmap_draft, norm =norm_draft)

    # Add contours
    # CD = axs[i][0].contour(newx, newy, newdraft,  [-300, -275, -250, -225, -200, -175, -150, -125, -100], cmap='inferno_r',zorder=1)
    # h, l = CD.legend_elements()

    axs[i][0].contour(newx, newy, -newdraft,  [100, 125, 150, 175, 200, 225, 250, 300], colors='black', linewidths=3, zorder=1)
    CD = axs[i][0].contour(newx, newy, -newdraft,  [100, 125, 150, 175, 200, 225, 250, 300], cmap='cmo.deep',  zorder=1)
    h, l = CD.legend_elements()

    # Set labels, and add year
    print((250000-xslicei[i])/2000 + xslicei[i]/1000)
    axs[i][0].set_ylabel('[km]', fontsize=11)
    axs[i][0].text(-90, (250000-xslicei[i])/2000 + xslicei[i]/1000, f'Model year {t}', fontsize=13,rotation=90,horizontalalignment='center',verticalalignment='center')
    axs[i][0].set_yticks([-250,-200,-150, -100,-50, 0, 50, 100, 150, 200, 250])
    
    ### COLUMN 2: MELT
    # Plot melt
    immelt = axs[i][1].pcolormesh( newx, newy, newmelt, cmap=cmap_BMB, norm = norm_BMB)

    # Print deepmelt and intmelt
    axs[i][1].text(38, xslicei[i]/1000+25, f'{deepmelt:.2f} m/yr', fontsize=11, horizontalalignment='right')
    axs[i][1].text(38, xslicei[i]/1000+5, f' {intmelt:.2f} Gt/yr', fontsize=11, horizontalalignment='right')

    print(f'max melt = {np.nanmax(xr.where(newdraft<-300, newmelt, np.nan).values)}')
    
    # Plot grounded and ocean for both melt and draft
    for j in range(2):
        axs[i][j].pcolormesh(newx, newy,  xr.where(newmask==1, 1, np.nan),      cmap=cmap_ocean,    norm=norm_ocean)
        axs[i][j].pcolormesh(newx, newy,    xr.where(np.logical_or(newmask==3,newmask==7), 1, np.nan),   cmap=cmap_ground,   norm=norm_ground)        

    ### COLUMN 3: PLUME VELOCITY
    # Plot plume velocity
    imvel = axs[i][2].pcolormesh( newx, newy, np.sqrt(newUt**2+newVt**2), cmap=cmap_u_lad, norm = norm_u_lad)
    # Mask ocean and grounded
    axs[i][2].pcolormesh(newx, newy,  xr.where(newmask==1, 1, np.nan),      cmap=cmap_ocean,    norm=norm_ocean)    
    axs[i][2].pcolormesh(newx, newy,    xr.where(np.logical_or(newmask==3,newmask==7), 1, np.nan),   cmap=cmap_ground,   norm=norm_ground)        
    
    # Show velocity vectors
    stepx = 5   # x spacing vectors
    stepy = 10  # y spacing vectors 
    # axs[i][2].quiver(newx[::stepx], newy[::stepy], newUt[::stepy,::stepx], newVt[::stepy, ::stepx], scale=1/0.6, width=1/150, facecolor='k')
    axs[i][2].quiver(newx[3::stepx], newy[::stepy], newUt[::stepy,3::stepx], newVt[::stepy, 3::stepx], scale=0.5, width=1/75, headlength=5, headaxislength=4, headwidth=3, facecolor='k')


    ### COLUMN 4: ICE VELOCITY
    imvelice = axs[i][3].pcolormesh( newx, newy, np.sqrt(newicevelu**2+newicevelv**2), cmap=cmap_u_surf, norm = norm_u_surf)
    
    # Mask ocean and GL
    axs[i][3].pcolormesh(newx, newy,  xr.where(newmask==1, 1, np.nan),      cmap=cmap_ocean,    norm=norm_ocean)        
    axs[i][3].pcolormesh(newx, newy,  xr.where(newmaskgl==1, 1, np.nan),      cmap=cmap_GL,    norm=norm_GL)

    # Show velocity vectors
    stepx = 5   # x spacing vectors
    stepy = 40  # y spacing vectors 
    axs[i][3].quiver(newx[::stepx], newy[::stepy], newicevelu[::stepy,::stepx], newicevelv[::stepy, ::stepx], scale=4000, width=1/100, facecolor='white')


# Set ticks and labels
for j in range(4):
    axs[2][j].set_xticks([-40,0,40])
    axs[1][j].set_xticks([-40,0,40])
    axs[0][j].set_xticks([-40,0,40])
    axs[2][j].tick_params(axis='both', labelsize=12)
    axs[1][j].tick_params(axis='both', labelsize=12)
    axs[0][j].tick_params(axis='both', labelsize=12)
    axs[2][j].xaxis.set_tick_params(labelbottom=True,labelcolor="k",)
    axs[2][j].set_xlabel('[km]', fontsize=11)

    # Set aspect
    axs[2][j].set_aspect(0.5)
    axs[1][j].set_aspect(0.5)
    axs[0][j].set_aspect(0.5)

# Take colors at regular intervals spanning the colormap.
colors = cmap_ground(np.linspace(0, 1, 17))
cplot=colors[0]

# Pannel labels
lab0 = ['(a)', '(b)', '(c)', '(d)']
lab200 = ['(e)', '(f)', '(g)', '(h)']
lab1000 = ['(i)', '(j)', '(k)', '(l)']
for j in range(3):
    axs[0][j].text(-37,20,lab0[j],fontsize=12,backgroundcolor=cplot,weight='bold')
    axs[1][j].text(-37,-90,lab200[j],fontsize=12,backgroundcolor=cplot,weight='bold')
    axs[2][j].text(-37,-240,lab1000[j],fontsize=12,backgroundcolor=cplot,weight='bold')

axs[0][3].text(-37,20,lab0[3],fontsize=12,c='white', weight='bold')
axs[1][3].text(-37,-90,lab200[3],fontsize=12,c='white', weight='bold')
axs[2][3].text(-37,-240,lab1000[3],fontsize=12,c='white', weight='bold')


# Set colorbars
cbar = plt.colorbar(immelt, ax = caxm, ticks=[-10,-1, 0, 1, 10, 100], label='sub-shelf melt [m/yr]', extend='both', orientation='horizontal', pad=.8, fraction=1)
cbar.ax.set_xticklabels([-10,-1, 0, 1, 10, 100], fontsize=11);
cbar.set_label('Sub-shelf melt \n[m/yr]', fontsize=11)

cbar = plt.colorbar(imvel,     ax = caxv, orientation='horizontal', label='layer velocity [m/s]',extend='max',  pad=.1, fraction=1,location='bottom')
cbar.ax.tick_params(labelsize=11)
cbar.set_label('Layer velocity \n[m/s]', fontsize=11)

cbar = plt.colorbar(imdraft,   ax = caxd, orientation='horizontal', label='draft [m]', extend='min', pad=.1, fraction=1)
cbar.ax.tick_params(labelsize=11)
cbar.set_label('Ice draft \n[m]', fontsize=11)

cbar = plt.colorbar(imvelice,   ax = caxi, orientation='horizontal', label='ice surface velocity [m/yr]', extend='max', pad=.1, fraction=1)
cbar.ax.tick_params(labelsize=11)
cbar.set_label('Ice surface velocity \n[m/yr]', fontsize=11)


caxm.set_axis_off()
caxv.set_axis_off()
caxd.set_axis_off()
caxi.set_axis_off()

for ax in axs[2]:
    ax.set_aspect(0.5)
# Set draft contour legend
axs[0][3].legend(h,['-100 m', '-125 m', '-150 m', '-175 m', '-200 m', '-225 m', '-250 m', '-300 m'], fontsize=11, bbox_to_anchor=(1.1, 1),loc=2)
# axs[0][0].legend(h,['-300 m', '-275 m', '-250 m', '-225 m', '-200 m', '-175 m', '-150 m', '-125 m', '-100 m'],bbox_to_anchor=(-1.1, 1),loc=2)

axs[0][0].text(-34  , 90,'GL'   , size=12 ,weight='bold',ha='center', va='center')
axs[0][0].text(-29  , 80,'w'    , size=9 ,weight='bold',ha='center', va='center')
axs[0][0].text( 33  , 90,'GL'   , size=12 ,weight='bold',ha='center', va='center')
axs[0][0].text( 38  , 80,'e'    , size=9 ,weight='bold',ha='center', va='center')

fig.align_ylabels() 

fig.savefig('figC1.pdf',dpi=300,bbox_inches='tight')

### Supplementary Cross-section:

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5,2), dpi=300)

ax.plot(la_M_IM.x/1000, la_M_IM.sel(time=0, y = 0).Hb, c='k', alpha=0.3, label='t = 0')
ax.plot(la_M_IM.x/1000, la_M_IM.sel(time=0, y = 0).Hs, c='k', alpha=0.3)
ax.plot(la_M_IM.x/1000, la_M_IM.sel(time=0, y = 0).Hs -la_M_IM.sel(time=0, y = 0).Hi , c='k', alpha=0.3)

ax.plot(la_M_IM.x/1000, la_M_IM.sel(time=200, y = 0).Hb, c='k', alpha=0.6, label='t = 200')
ax.plot(la_M_IM.x/1000, la_M_IM.sel(time=200, y = 0).Hs, c='k', alpha=0.6)
ax.plot(la_M_IM.x/1000, la_M_IM.sel(time=200, y = 0).Hs -la_M_IM.sel(time=200, y = 0).Hi , c='k', alpha=0.6)

ax.plot(la_M_IM.x/1000, la_M_IM.sel(time=1000, y = 0).Hb, c='k', label='t = 1000')
ax.plot(la_M_IM.x/1000, la_M_IM.sel(time=1000, y = 0).Hs, c='k')
ax.plot(la_M_IM.x/1000, la_M_IM.sel(time=1000, y = 0).Hs -la_M_IM.sel(time=1000, y = 0).Hi , c='k')

ax.set_xlabel('X [km]')
ax.set_ylabel('[m]')
ax.legend()

fig.tight_layout()

fig.savefig('figS3.pdf',dpi=300,bbox_inches='tight')